In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
shelter = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invalid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will return a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

#Add in Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1("Christian Henshaw Global Rain Dashboard"))),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        dcc.RadioItems(
        id='filter-type',
        options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster or Individual Tracking Rescue', 'value': 'DITR'},
            {'label': 'Reset Filter', 'value': 'RESET'}
        ],
            value='RESET',
            inline=True
        )
    ),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_columns=[],
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=5
    ),
    html.Br(),
    html.Hr(),
    #Sets up dashboard so histogram and geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(Output('datatable-id','data'),
              Input('filter-type', 'value'))
def update_dashboard(filter_type):
    #default state
    if filter_type == 'RESET':
        df = pd.DataFrame.from_records(shelter.read({}))
    #searches for potential water rescue dogs
    elif filter_type == 'WR':
        df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", 
                                                     "breed":{"$in":["Labrador Retriever Mix",
                                                                     "Chesapeake Bay Retriever", 
                                                                     "Newfoundland Mix",
                                                                     "Newfoundland/Labrador Retriever",
                                                                     "Newfoundland/Australian Cattle Dog",
                                                                     "Newfoundland/Great Pyrenees"]},
                                                    "sex_upon_outcome": "Intact Female",
                                                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}))
    #searches for potential mountain or wilderness rescue dogs
    elif filter_type == 'MWR':
        df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", 
                                                     "breed":{"$in":["German Shepherd",
                                                                     "Alaskan Malamute", 
                                                                     "Old English Sheepdog",
                                                                     "Siberian Husky",
                                                                     "Rottweiler"]},
                                                    "sex_upon_outcome": "Intact Male",
                                                    "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}}))
    #searches for disaster or individual tracking rescue dogs
    elif filter_type == 'DITR':
        df = pd.DataFrame.from_records(shelter.read({"animal_type": "Dog", 
                                                     "breed":{"$in":["German Shepherd",
                                                                     "Doberman Pinscher", 
                                                                     "Golden Retriever",
                                                                     "Bloodhound",
                                                                     "Rottweiler"]},
                                                    "sex_upon_outcome": "Intact Male",
                                                    "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}}))
    df.drop(columns=['_id'],inplace=True)
    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    #import selected data
    dff = pd.DataFrame.from_dict(viewData)
    #identify values needed to display
    names = dff["breed"].value_counts().keys().tolist()
    counts = dff["breed"].value_counts().tolist()
    return [
        dcc.Graph(
            figure = px.histogram(dff, x = names, y = counts, title = "Preferred Animals", nbins = 20))
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9]),
                    html.H1("Age in Weeks"),
                    html.P(dff.iloc[row,15]),
                    html.H1("Breed"),
                    html.P(dff.iloc[row,4])
                ])
            ])
        ])
    ]



app.run_server(debug=True)


Connection Established!
Dash app running on http://127.0.0.1:17570/
